In [2]:
pip install dash plotly pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [4]:
import dash
from dash import dcc, html, Input, Output, dash_table
import plotly.express as px
import pandas as pd
import numpy as np

In [9]:
df=pd.read_csv('data_assign.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              8 non-null      object
 1   Personality Type  8 non-null      object
 2   Score             8 non-null      int64 
 3   Date              8 non-null      object
dtypes: int64(1), object(3)
memory usage: 388.0+ bytes


In [11]:
df.head()

,Name,Personality Type,Score,Date
0,Pant,Extrovert,85,30/01/2025
1,Bumrah,Introvert,90,30/01/2025
2,Jadeja,Ambivert,78,30/01/2025
3,Rohith,Extrovert,88,30/01/2025
4,Kohli,Ambivert,92,30/01/2025


In [13]:
df['Rank'] = df['Score'].rank(ascending=False, method='min').astype(int)

In [14]:
def get_improvement_suggestion(personality_type):
    if personality_type == 'Introvert':
        return "Consider joining group activities to improve social skills."
    elif personality_type == 'Extrovert':
        return "Focus on listening skills and self-reflection."
    elif personality_type == 'Ambivert':
        return "Balance social interactions with alone time."
    return "No suggestion available."

In [15]:
df['Improvement Suggestions'] = df['Personality Type'].apply(get_improvement_suggestion)

In [16]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Personality Assessment Leaderboard", style={'textAlign': 'center'}),
    html.Label("Filter by Personality Type:"),
    dcc.Dropdown(
        id='personality-filter',
        options=[{'label': i, 'value': i} for i in df['Personality Type'].unique()],
        multi=True,
        placeholder="Select Personality Type(s)"
    ),
    dash_table.DataTable(
        id='leaderboard-table',
        columns=[
            {'name': 'Rank', 'id': 'Rank'},
            {'name': 'Name', 'id': 'Name'},
            {'name': 'Personality Type', 'id': 'Personality Type'},
            {'name': 'Score', 'id': 'Score'},
            {'name': 'Improvement Suggestions', 'id': 'Improvement Suggestions'}
        ],
        data=df.to_dict('records'),
        style_data_conditional=[
            {
                'if': {'filter_query': '{Rank} <= 3'},
                'backgroundColor': 'gold',
                'color': 'black'
            }
        ],
        style_header={
            'backgroundColor': 'lightblue',
            'fontWeight': 'bold'
        },
        style_cell={
            'textAlign': 'left',
            'padding': '10px'
        }
    ),
    html.Div([
        dcc.Graph(id='bar-chart'),
        dcc.Graph(id='pie-chart')
    ], style={'display': 'flex', 'justifyContent': 'space-around'})
])

In [17]:
@app.callback(
    [Output('leaderboard-table', 'data'),
     Output('bar-chart', 'figure'),
     Output('pie-chart', 'figure')],
    [Input('personality-filter', 'value')]
)
def update_dashboard(selected_types):
    filtered_df = df if not selected_types else df[df['Personality Type'].isin(selected_types)]
    table_data = filtered_df.to_dict('records')
    bar_fig = px.bar(
        filtered_df,
        x='Personality Type',
        y='Score',
        title='Personality Type Distribution',
        color='Personality Type',
        labels={'Score': 'Average Score'}
    )
    pie_fig = px.pie(
        filtered_df,
        names='Personality Type',
        title='Personality Type Percentage',
        hole=0.3
    )
    return table_data, bar_fig, pie_fig

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>